In [ ]:
!pip install pandas 
!pip install mysql-connector-python
!pip install tqdm




In [2]:
import os
import csv
import mysql.connector
from tqdm import tqdm

# in this script, I have written it in that after you've extracted the datasets
# first, if you are using XAMPP, open the control panel and start mysql and apache servers
# other applications you may search on how to start your mysql
# if youve installed anaconda: -
# i. search for anaconda prompt on start menu
# ii. start it and type "jupyter notebook"
# iii. it will open in your browser a directory
# iv. select this file, maybe iko in the downloads
# run each cell, the last cell will load the datasets in your mysql database
# few things to note
# kama you have set a user and password while installing your mysql ::::
# # 
# Database connection details
# host = 'localhost' // just means you are in your local directory dont adjust
# user = 'your_username' // the one you set during installation
# password = 'your_password' // the one you set for the username
### Also, path to your file
# # place the path to your extracted dataset folder so that it can loop the csv files
# folder_path = 'D:/banking_data1/banking_data1' // Edit hapa ikue the path to the extracted dataset yako
# this line iko chini ya the code
# so basically only the two parts are edited as for the database name I've set it to 'banking_details' for uniformity and server access
# we can use the same
# the code runs for a while because of how large some of the csv files are but nimeeka the progress bar for context 
# any other queries you may ask
# Enjoy

In [ ]:
# 
# Database connection details
host = 'localhost'
user = 'root'
password = ''

def create_database(database_name):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )

    cursor = connection.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{database_name}`")

    connection.close()

def create_table(database_name, table_name, header):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database_name
    )

    cursor = connection.cursor()
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` (id INT AUTO_INCREMENT PRIMARY KEY, "

    for column in header:
        create_table_query += f"`{column}` VARCHAR(255), "

    create_table_query = create_table_query.rstrip(', ') + ")"
    cursor.execute(create_table_query)

    connection.close()

def insert_data(database_name, table_name, header, rows):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database_name
    )

    cursor = connection.cursor()
    insert_query = f"INSERT INTO `{table_name}` (`{'`, `'.join(header[1:])}`) VALUES ({', '.join(['%s'] * (len(header) - 1))})"

    with tqdm(total=len(rows), unit='row') as pbar:
        for row in rows:
            cursor.execute(insert_query, row[1:])
            pbar.update(1)

    connection.commit()
    connection.close()

def select_data(database_name, table_name, limit=5):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database_name
    )

    cursor = connection.cursor()
    select_query = f"SELECT * FROM `{table_name}` LIMIT {limit}"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    connection.close()
    return rows

def main(folder_path):
    database_name = 'banking_details'
    csv_files = []  # List to store the directory paths of CSV files

    # Iterate through the files in the folder
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):  # Check if the file is a CSV file
            file_path = os.path.join(folder_path, file)  # Get the full file path
            csv_files.append(file_path)  # Add the file path to the list

    if not csv_files:
        print("No CSV files found in the folder.")
        return

    # Create the database if it doesn't exist
    create_database(database_name)

    for csv_file in csv_files:
        table_name = os.path.splitext(os.path.basename(csv_file))[0]

        # Read the CSV file and retrieve the header and rows
        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            header = next(reader)
            rows = list(reader)

        # Create the table with the header columns
        create_table(database_name, table_name, header)

        # Insert the data into the table
        insert_data(database_name, table_name, header, rows)

        # Select and display the first five rows from the table
        selected_rows = select_data(database_name, table_name, limit=5)
        print(f"First five rows from the table '{table_name}':")
        for row in selected_rows:
            print(row)

    print("Process completed successfully.")

# place the path to your extracted dataset folder so that it can loop the csv files
folder_path = 'yourdatalocation'
if not os.path.exists(folder_path):
    print("Please check the folder path.")
else:
    main(folder_path)
